In [3]:
import tensorflow as tf

# Habilita el crecimiento de memoria para todos los dispositivos GPU antes de inicializar el runtime
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Actualmente, la configuración de crecimiento de memoria solo está disponible en CUDA
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        # Si ya se ha inicializado el runtime, se producirá un error al intentar establecer el crecimiento de memoria.
        print(e)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Especifica la GPU a utilizar (0 para la primera GPU)
import cupy as cp

In [4]:
import tensorflow as tf
import platform

print("TensorFlow version:", tf.__version__)
print("TensorFlow is running on:", platform.platform())

# Obtén la versión de CUDA
devices = tf.config.list_physical_devices('GPU')

TensorFlow version: 2.9.0
TensorFlow is running on: Windows-10-10.0.22631-SP0


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
#import seaborn as sns
import os
#from scipy.sparse import hstack
#import funciones_ML as bt
from sklearn.feature_extraction.text import TfidfVectorizer, FeatureHasher
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler, LabelEncoder, Normalizer,MaxAbsScaler
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay,classification_report,r2_score,RocCurveDisplay,confusion_matrix, accuracy_score,recall_score,f1_score,precision_score,precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, SGDClassifier, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

#from imblearn.under_sampling import RandomUnderSampler

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint,ReduceLROnPlateau,CallbackList
from tensorflow.keras.regularizers import l2, l1, l1_l2
from tensorflow.keras import regularizers
from tensorflow.keras.losses import CategoricalCrossentropy, sparse_categorical_crossentropy, binary_crossentropy
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers.experimental import preprocessing


import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
# Ruta al archivo CSV gigante
file_path_wsl = "/mnt/d/Cursos/REPOSITORIOS/DATASET/malware_total/original/df_one_hot_full_ml_mm.csv"
file_path_w11= "D:\Cursos\REPOSITORIOS\DATASET\malware_total\original\df_one_hot_full_ml_mm.csv"


In [8]:
import dask.dataframe as dd
import dask.array as da
import dask.distributed
import dask
import cupy as cp
from dask.distributed import LocalCluster, Client
from distributed import Client
import threadpoolctl
from dask.delayed import Delayed
from sklearn.utils import check_random_state
import ray.data as rd
import ray
import ray.data
import dask
from dask_ml.wrappers import Incremental
from dask_ml.decomposition import PCA
from dask_ml.datasets import make_regression
import vaex
from dask.diagnostics import ProgressBar
import tables
import h5py
import tempfile

In [ ]:
import cupy as cp
import dask.array as da

# Crear un array de Dask respaldado por CuPy
x_gpu = da.from_array(cp.ones((10000, 10000)), chunks=(1000, 1000))

# Realizar una operación de reducción en paralelo en la GPU
sum_gpu = x_gpu.sum().compute()

print(sum_gpu)

In [17]:
from tqdm import tqdm

# Función para simular una tarea que tarda un tiempo
def tarea_larga(n):
  for i in tqdm(range(n), desc="Procesando datos...", unit="iteraciones", total=n, leave=True):
    time.sleep(0.1)




In [10]:
df=ray.data.read_csv(file_path_w11)

In [11]:
df.schema()

Column                Type
------                ----
                      int64
0                     double
1                     double
2                     double
3                     double
4                     double
5                     double
6                     double
proto_icmp            double
proto_tcp             double
proto_udp             double
conn_state_OTH        double
conn_state_REJ        double
conn_state_RSTO       double
conn_state_RSTOS0     double
conn_state_RSTR       double
conn_state_RSTRH      double
conn_state_S0         double
conn_state_S1         double
conn_state_S2         double
conn_state_S3         double
conn_state_SF         double
conn_state_SH         double
conn_state_SHR        double
history_Aa            double
history_Ar            double
history_C             double
history_CCC           double
history_CCCC          double
history_D             double
history_DAd           double
history_DFafA         double
history_DFr        

In [10]:
with ProgressBar():
    df = dask.dataframe.read_csv(file_path_w11, assume_missing=True)
    
df = df.drop('6', axis=1)

# Convertir todas las columnas a numéricas
for col in df.columns:
    df[col] = df[col].apply(pd.to_numeric, errors='coerce',  meta=(col, 'float64'))

# Rellenar los valores NaN con 0
df = df.fillna(0)

# Convertir el DataFrame de Dask a un array de Dask
array = df.to_dask_array(lengths=True)

# Inicializar una lista para almacenar los nombres de los archivos temporales
temp_files = []

# Iterar sobre los chunks del array
for chunk in array.to_delayed().flatten():
    # Convertir el chunk a un array de CuPy
    chunk = cp.asarray(chunk.compute())
    
    # Realizar la PCA en el chunk
    u, s, vh = cp.linalg.svd(chunk, full_matrices=False)
    
    #liberacion de memoria
    cp._default_memory_pool.free_all_blocks()
    
    # Convertir los arrays a numpy
    u = u.get()
    s = s.get()
    vh = vh.get()

    
    # Crear un archivo temporal para guardar los resultados de PCA
    temp_file = tempfile.NamedTemporaryFile(dir=r"D:\Cursos\REPOSITORIOS\DATASET\malware_total\original\dir",delete=False)
    
     # Cerrar el archivo temporal antes de guardar los datos en él
    temp_file.close()
    
     # Guardar los resultados de PCA en el archivo temporal
    np.savez(temp_file.name, u=u, s=s, vh=vh)
    
    #Añadir el nombre del archivo temporal a la lista
    temp_files.append(temp_file.name)
    
    # Liberar memoria de la GPU
    cp._default_memory_pool.free_all_blocks()

# Inicializar una lista para almacenar los resultados de PCA
pca_results = []

# Cargar los resultados de PCA de los archivos temporales
for temp_file in temp_files:
    try:
        with np.load(temp_file) as data:
            u = data['u']
            s = data['s']
            vh = data['vh']
            pca_results.append((u, s, vh))
    except Exception as e:
        print(f"No se pudo cargar el archivo {temp_file} debido al error: {e}")
    
    # Intentar eliminar el archivo temporal
    try:
        os.remove(temp_file)
    except PermissionError:
        print(f"No se pudo eliminar el archivo {temp_file}. Continuando con el siguiente archivo.")
        
# Convertir la lista de resultados de PCA a un array de Dask respaldado por CuPy
pca_results = da.from_array(cp.asarray(pca_results), chunks=(500, 500))

# Realizar una operación de reducción en paralelo en la GPU
pca_combined = pca_results.sum().compute()

# Obtener las características (X) excluyendo la columna de la etiqueta
X = pca_combined[:, :-1]

# Obtener la etiqueta (y) que es la última columna
y = pca_combined[:, -1]

OutOfMemoryError: Out of memory allocating 124,156,416 bytes (allocated so far: 0 bytes).

In [6]:
df = dask.dataframe.read_csv(file_path_w11, assume_missing=True, blocksize=1_000_000)


In [5]:
X = df.drop(columns=["6"])
y = df["6"]

In [6]:
pca = PCA(n_components=100, svd_solver='randomized')
X_pca = pca.fit(X).transform(X)

In [ ]:
# Realiza el split en conjunto de entrenamiento y conjunto de pruebas utilizando train_test_split de sklearn:
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)

In [4]:
df = dask.dataframe.read_csv(file_path_w11, assume_missing=True, blocksize=None)

# Ajustar el número de componentes a conservar en PCA
n_components = 140

# Realizar PCA
pca = PCA(n_components=n_components, svd_solver='randomized', random_state=0)
X = df.drop(columns=[df.columns[6]])
Y = df[df.columns[6]].astype(np.float32)
X_pca = pca.fit_transform(X)

# Generar conjuntos de datos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)

# Guardar el DataFrame en un archivo CSV
df.to_csv("D:\Cursos\REPOSITORIOS\DATASET\malware_total\original\PCA_df_one_hot_full_ml_mm.csv", index=False)
